In [237]:
import pandas as pd
import numpy as np
import requests

import json

pd.options.display.max_rows = 400
pd.options.display.max_colwidth = 4000

In [2]:
df_searches = pd.read_excel("searches_candidates_selection_rank.xlsx")
df_searches = df_searches[df_searches['query'] != 'undefined']

In [3]:
def slug_only(dr):
    candidates = json.loads(dr.replace("'", '"'))    
    try :
        return json.dumps([c['slug'] for c in candidates])
    except TypeError:
        print('ERROR:\n' + dr)
        return None

df_searches['docs_slug'] = df_searches.docs_res.apply(lambda dr : slug_only(dr))

ERROR:
['fiches_ministere_travail/le-registre-unique-du-personnel#Quelle-forme-doit-revetir-le-registre-unique-du-personnel', 'fiches_service_public/registres-du-personnel-obligatoires', 'fiches_ministere_travail/le-registre-unique-du-personnel#Les-stagiaires-sont-ils-concernes', 'fiches_ministere_travail/le-registre-unique-du-personnel#Quelles-sont-les-sanctions-en-cas-d-infraction', 'fiches_ministere_travail/le-registre-unique-du-personnel#Quels-sont-les-employeurs-concernes', 'fiches_service_public/lanceur-dalerte-dans-lentreprise-quelles-sont-les-regles', 'fiches_ministere_travail/le-registre-unique-du-personnel#Pour-quels-salaries']


In [4]:
df_searches_unique = df_searches[["rank", "docs_slug", "query", "docs_res", "idVisit"]]
df_searches_unique.drop_duplicates(["docs_slug", "query", "idVisit"], inplace=True)

/Users/remim/dev/cdtn/.python-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df_searches.shape

(117817, 9)

# Evaluation : group queries and get rank for each candidates 
Objective : compute NDCG / MAP / MRR

In [6]:
# most frequent queries with hit
all_ranks = []

threshold = 5
for g, rows in df_searches_unique[df_searches['rank'] >= 0].groupby(["docs_slug"]):
    try:
        n = rows.shape[0]
        q = set(rows['query'].tolist())    
        algos = [docres['algo'] for docres in json.loads(rows.docs_res.tolist()[0].replace("'",'"'))]
        #algos = 
        preq = 'pre-qualified' in algos
        candidates = g
        if (n > threshold):        
            print(f'{q} -> {n}')
            #print(algo)
            ranks = rows['rank'].value_counts().sort_index()
            candidates_array = json.loads(candidates)
            j = len(candidates_array)
            complete_ranks = [0] * j
            ids = []
            for i in range(0, j):
                if i in ranks.index:
                    complete_ranks[i] = ranks[i]             
                ids.append({'slug':candidates_array[i], 'count':int(complete_ranks[i]), 'preq':algos[i]})
            all_ranks.append({'total':int(n), 'queries':list(q), 'preq': preq, 'docs':ids})
    except KeyError:
        print("Key error : " + g)
        continue

/Users/remim/dev/cdtn/.python-env/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


{'indemnité de départ en retraite', 'comment calculer les indemnités de départ à la retraite ?', 'PRIME DE DEPART A LA RETRAITE', 'Indemnités de départ à la retraite', 'indemnité de depart à la retraite', 'indemnité de départ à la retraite', 'quel est le montant de la prime de départ à la retraite ?', 'indemnité départ retraite', 'prime de depart retraite', 'Indemnité départ à la retraite', 'montant des indemnites de depart a la retraite', 'indemnité départ à la retraite', 'prime depart a la retraite', 'prime de départ à la retraite'} -> 153
{'indemnités'} -> 19
{"maladie pendant période d'essai", "arrêt maladie pendant période d'essai", "prolongation période d'essai accident du travail"} -> 11
{'arrêt maladie préavis', "l'arrêt maladie prolonge t il un préavis"} -> 11
{'RENOUVELLEMENT CDD', 'Renouvèlement cdd', 'renouvellements cdd', 'Renouvellement cdd', 'renouvellement cdd', 'prolongation cdd', 'renouveler cdd', 'renouveler CDD'} -> 57
{"ancienneté et congé parental d'éducation", 'A

{'invalidite', 'invalidité'} -> 8
{'handicap'} -> 7
{'travailleur handicape', 'travailleur handicapé'} -> 6
{'Temps partiel'} -> 11
{'temps partiel', 'TEMPS PARTIEL'} -> 122
{'horaire', 'horaires'} -> 14
{'demande de temps partiel', 'Demande temps partiel', 'demander à travailler à temps partiel', 'passage temps partiel', 'passage à temps partiel'} -> 8
{'saisie sur salaire'} -> 7
{"prime d'activité"} -> 7
{'indemnités chômage', 'droit au chômage', 'chômage', 'quel type de licenciement pour toucher le chômage ?', 'indemnisation chômage', 'Indemnite chômage', 'Chomage', 'ai je droit au chômage', 'allocation chomage', 'Droit au chomage', 'chomage', 'indemnités pôle emploi', 'Droit chômage', 'conditions pour avoir le droit au chômage', 'indemnité chomage', 'Chômage', 'allocation chômage et rupture du contrat de travail cdd', 'allocation chômage', 'allocation de chomage', 'DOIT AU CHOMAGE', 'droits au chomage', 'allocation chômage et rupture du contrat de travail', 'allocations de chômage'

{"calcul d'indemnité pour motif économique"} -> 7
{'dpae'} -> 10
{'démission légitime', 'Démission légitime', 'demission légitime', 'Démission legitime', 'demission legitime'} -> 9
{'demission cdd', 'démission CDD', 'démission cdd', 'DEMISSION CDD', 'Démission CDI', "rompre le cdd à l'initiative du salarié", 'Démission CDD', 'rupture cdd a l initiative du salarié', 'Démission cdi', 'un salarié peut-il démissionner dans le cadre d’un cdd ?', 'démission cdi'} -> 81
{'temps de repos légal'} -> 8
{'duree du travail', 'durée du travail', 'durée de travail', 'duréé de travail', 'duréé du travail'} -> 49
{'durées maximales du travail', 'durée maximale de travail', 'durée maximale du travail'} -> 28
{'temps de travail effectif par mois', 'TEMPS DE TRAVAIL EFFECTIF', "régime d'équivalence et temps de travail effectif", 'Temps de travail effectif', 'calcul temps de travail effectif', 'astreinte et temps de travail effectif', 'formation et temps de travail effectif', 'temps travail effectif', 'te

{'Nuit', 'nuit'} -> 9
{'heure de nuit', 'Heure de nuit', 'heure nuit'} -> 7
{'heures de nuit', 'quelles sont les heures de nuit', 'heures nuit'} -> 8
{'dimanche'} -> 6
{'cdd saisonnier', 'Cdd saisonnier'} -> 7
{'contrat saisonnier'} -> 6
{'index égalité professionnelle', 'index ega pro', 'égalité professionnelle index', 'index égalité professionnelle entre les femmes et les hommes', 'modèle accord égalité professionnelle hommes femmes 2019'} -> 10
{'absence'} -> 8
{'congé enfant malade', 'conge enfant malade'} -> 120
{'congé enfant malade'} -> 11
{'enfant malade'} -> 18
{'enfants malades', 'Enfant malade'} -> 17
{'maladie'} -> 29
{'indemnites de conges payes', 'indemnité de congés payes', 'indemnité de congés payés', 'indemnités de congés payés'} -> 8
{'indemnite conges payes', 'indemnités congé payé', 'indemnite congés payés', 'indemnités congés payés', 'indemnité congés payés'} -> 10
{'indemnités congé payés'} -> 19
{'don de congés payés', 'conges payes assiette', 'alternance et cong

...

{'local aveugle réglementation', 'lieu de travail différent du lieu de rattachement', 'lieu de travail contrat de travail', 'dimension bureau de travail', 'Lieu de travail', 'lieu de travail', 'conception des lieux de travail', "surfaces minimales d'un espace de travail"} -> 40
{'faute du salarié', 'quels sont les types de fautes professionnelles ?', "mon salarié m'a insulté", 'faute professionnelle', 'quel délai pour sanctionner une faute ?', 'manquement aux règles professionnelles faute professionnelle', 'quelle sanction pour faute professionnelle ?'} -> 34
{'licenciement pour faute', 'licenciement pour faute grave', 'licenciement faute grave', 'Licenciement pour faute'} -> 22
{'licenciement faute grave', 'Licenciement pour faute', 'licenciement faute lourde', 'liceicement pour faute grave', 'Licenciement pour faute grave', 'licenciement pour mauvaise conduite', 'licenciement pour faute lourde', 'LICENCIEMENT POUR FAUTE', 'licenciement pour faute grave', 'licenciement pour faute', 'L

{'rôle médecin du travail'} -> 9
{'médecin du travail', 'Médecine du travail', 'medecin du travail'} -> 6
{'souffrance au travail', 'Burn-out que faire', 'Au secours', 'burn out', 'burn out que faire', 'burn out travail', 'demande de renseignements', 'arrêt maladie burn out', 'Burn-out', 'Souffrance au travail'} -> 50
{'Avenant contrat travail', 'avenant au contrat de travail', 'avenant de contrat', 'Avenant au contrat de travail', 'avenant contrat de travail', 'avenant contrat travail', 'avenant au contrat', 'Avenant de contrat'} -> 84
{'modele contrat de travail', 'modification contrat de travail refus', 'changement de contrat de travail', 'modification contrat de travail', 'Changement dans le contrat de travail', 'changement du contrat de travail', 'modification du contrat de travail', 'modification contrat', 'modifier mon contrat de travail'} -> 58
{"modifier le temps de travail d'un salarié", "l'employeur peut il imposer au salarié la réduction des horaires", "modification du cont

{'ijss arrêt de travail et accident du travail', 'acident du travail', 'accident travail', 'Accident travail', 'accident de travil', 'accident du travail prise en charge', 'Accident de travail', 'Accidents du travail', 'accident du travail', 'Accident de travail et congés payés', 'arrêt de travail suite à un accident du travail', 'indemnisation accident du travail', 'Accidents de travail', 'arrêt de travail et accident du travail', 'accidents du travail', 'accident de travail', 'Accident du travail', 'ACCIDENT DU TRAVAIL', 'Accidon de travail'} -> 183
{'clause de mobilité et refus du salarié'} -> 8
{'clause de mobilité', 'qu’est-ce qu’une clause de mobilité ?'} -> 31
{'insuffisance professionnelle'} -> 7
{'imaldie professionnelle', 'maladie professionnelle', 'maladie professionnel', 'Maladie professionnele', 'maladie professionnelle et rupture du cdd', 'maladie professionnnelle', 'maladie profesionnelle', 'maladie%20professionnelles', 'maladies professionnelles', 'maladie proffessionne

{'annualisation temps travail', 'comment sont calculées les 1607 heures ?', "c'est quoi la modulation du temps de travail ?", "qu'est-ce qu'un temps partiel modulé ?", 'modulation du temps de travail', 'comment annualiser un temps de travail ?', 'annualisation temps de travail', 'Annualisation du temps de travail', 'annualisation du temps de travail', 'aménagement du temps de travail', 'Annualisation temps de travail', "comment calculer l'annualisation du temps de travail ?", 'modulation du temps de travail et congés payés', 'annualisation du temps de travail et congés payés', 'annualisation du temps de travail temps partiel', 'aménagement temps de travail', 'ANNUALISATION DU TEMPS DE TRAVAIL'} -> 132
Key error : ["/fiche-service-public/temps-partiel-dans-le-secteur-prive", "/fiche-service-public/lemployeur-peut-il-instaurer-du-travail-a-temps-partiel-dans-lentreprise", "/fiche-service-public/comment-faire-pour-passer-a-temps-partiel-dans-le-secteur-prive", "/fiche-ministere-travail/le

{'lettre de contestation, par le salarié, du refus de la demande de congés payés', 'refus de congés abusif', 'refus conges payé', "Refus de congés payés par l'employeur", 'délai de prévenance refus congés payés', 'refus de congés payés par le salarié', 'quand un employeur peut refuser des congés ?', 'refus congés payés', "reports de congés payés par l'employeur", 'refus congés payés après congé maternité', 'est-il obligatoire de prendre ses congés payés ?', "refus de congés payés par l'employeur", 'délai de prévenance congés payés'} -> 78
Key error : ["/modeles-de-courriers/releve-dheures-supplementaires", "/fiche-service-public/heures-supplementaires-dun-salarie-du-secteur-prive", "/fiche-service-public/heures-dequivalence-dans-le-secteur-prive", "/fiche-service-public/astreinte-dans-le-secteur-prive", "/fiche-service-public/astreinte-dans-le-secteur-prive", "/fiche-ministere-travail/les-heures-supplementaires-definition-et-limites#Quel-est-le-cadre-de-decompte-des-heures-supplementai

In [7]:
print(len(all_ranks))
#print()\
#print(sorted(all_ranks[6], key=lambda x: x['count'], reverse=True))

585


In [ ]:
print(json.dumps(all_ranks[15], indent=2))

In [ ]:
# from documentation

params = {
    "y_pred" : ["a", "b", "c", "w", "k","e"], # y_pred (array): list of documents id predicted by the system
    "y_true" : ["a", "b", "c", "e"], # y_true (array): documents id scored by humans sorted from most relevant to least relevant
    "y_score" : {
        "a":5,
        "b":3,
        "c":2,
        "e":2
            }, # y_score is a dict {"doc_id":"score"} of documents assigned as relevant y humans with the associated scores (ordered internally)
    "method" : "all", # one of ["precision", "recall", "dcg", "mrr", "all"]
    "k": 3 # k integer preferaly =< to length(y_pred)
}

In [8]:
def sort_ranks(ranks):
    return sorted(ranks, key=lambda x: x['count'], reverse=True)


In [8]:
# not used anymore
'''
def compute_metrics(rank_results, i, verbose=False, api=False):
    scores = dict([(r['slug'], int(r['count'])) for r in rank_results[i]['docs']])
    if not api:
        y_pred = [c['slug'] for c in rank_results[i]['docs']]
    elif api:
        y_pred = rank_results[i]['docs_api']
        
    params = {

        "y_pred": y_pred,
        #"y_pred": [c['slug'] for c in sort_ranks(all_ranks[i]) if c['count'] > 0],
        "y_true" : [c['slug'] for c in sort_ranks(rank_results[i]['docs']) if c['count'] > 0],
        "y_score" : scores,
        "method" : "all",
        "k":5
    }
    r = requests.post("http://localhost:4545/api/score", json = params)
    
    if r.status_code != 200:
        print("Error with at index " + str(i))        
        result = {'dcg': 1.0, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}
    else :
        result = r.json()
        
    if verbose:
        print(i)
        print(rank_results[i][0]['queries'])
        print(json.dumps(scores, indent=2))
        print(result)
    
    result['occ'] = rank_results[i]['total']
    return result
'''

'\ndef compute_metrics(rank_results, i, verbose=False, api=False):\n    scores = dict([(r[\'slug\'], int(r[\'count\'])) for r in rank_results[i][\'docs\']])\n    if not api:\n        y_pred = [c[\'slug\'] for c in rank_results[i][\'docs\']]\n    elif api:\n        y_pred = rank_results[i][\'docs_api\']\n        \n    params = {\n\n        "y_pred": y_pred,\n        #"y_pred": [c[\'slug\'] for c in sort_ranks(all_ranks[i]) if c[\'count\'] > 0],\n        "y_true" : [c[\'slug\'] for c in sort_ranks(rank_results[i][\'docs\']) if c[\'count\'] > 0],\n        "y_score" : scores,\n        "method" : "all",\n        "k":5\n    }\n    r = requests.post("http://localhost:4545/api/score", json = params)\n    \n    if r.status_code != 200:\n        print("Error with at index " + str(i))        \n        result = {\'dcg\': 1.0, \'mrr\': 1.0, \'precision\': 1.0, \'recall\': 1.0}\n    else :\n        result = r.json()\n        \n    if verbose:\n        print(i)\n        print(rank_results[i][0][\'que

In [140]:
# Borrowed from Armand's letor
def discounted_cumulative_gain(y_score, y_true, y_pred, k):
    """y_score is a dictionnary {"doc_id":"score"} of documents assigned as relevant y humans with the associated scores
       k = maximum index to be scored
       /!\ to be valid, dcg should have results lists of same length bbetween requests"""
    if y_pred == []:
        return 0
    if k > len(y_pred):
        k = len(y_pred)
    
    y_scoring = []
    for y in y_pred:
        score = y_score.get(y)
        if score is None: # if the predicted document is not in the array of humanly scored documents
            score = 0
        y_scoring.append(score)
    
    # compute dcg
    def compute_dcg(y_scoring, k):
        dcg = []
        for i, pred in enumerate(y_scoring[:k]):
            i+=1
            # two formulas possible, according to wikipedia the latter places stronger emphasis on retrieving relevant documents
            dcg.append(pred/(np.log2(i)+1))
            #dcg.append((np.power(pred,2) - 1)/(np.log2(i)+1))
        return np.sum(dcg)
    # compute Ideal DCG
    # ideal DCG: the best score that could have been obtained given the relevant document list
    # i. e. : the most relevant documents, ordered by relevance.

    dcg = compute_dcg(y_scoring, k)

    ideal_scores = []
    for i in y_true:
        score = y_score.get(i)
        if not i in y_score : # if we don't have a score for y true
            y_score[i]
            raise ValueError("the true document does not have score in y_score : " + i)
        ideal_scores.append(score)

    ideal_scores = sorted(ideal_scores, reverse=True)

    ideal_dcg = compute_dcg(ideal_scores, k)
    return dcg/ideal_dcg

# Update candidates with API

In [ ]:
# for each group, trigger query 
all_ranks[113]

In [ ]:
all_scores.iloc[113]

In [10]:
#[ar for ar in range(len(all_ranks)) if 'préavis cdd' in all_ranks[ar]['queries']]

In [120]:
def reformat_slug(source, slug) :
    formated_source = source.replace('_', '-', )
    #print(formated_source)
    #print(source)
    formated_source = formated_source.replace('fiches', 'fiche')
    if (source == 'contributions'):
        formated_source = formated_source[:-1]
        
    return f"/{formated_source}/{slug}"

def call_api(query):
    r = requests.get("http://localhost:1337/api/v1/search?q=" + query)
    if r.status_code != 200:
        print("Error when calling search API for query : " + str(q))        
        return {}
    else :
        result = r.json()
        return result

In [121]:
def control_call(i):
    try:
        api_res = call_api(all_ranks[i]['queries'][0])['documents']       
        call_res = [reformat_slug(d['source'], d['slug']) for d in api_res]
        origin_res = [d['slug'] for d in all_ranks[i]['docs']]
        #if (call_res != origin_res):
        #    return False
            #print([call_res[j] == origin_res[j] for j in range(len(call_res))])
            #print(json.dumps(call_res, indent=2))
            #print(json.dumps(origin_res, indent=2))

        #else:
        all_ranks[i]['docs_api'] = call_res
        return True
    except KeyError:
        return False
    except IndexError:
        return False


In [150]:
j = 0
issues = []
for i in range(0, len(all_ranks)):
#for i in range(0, 2):
    if not control_call(i):
        issues.append(i)
len(issues)

4

In [ ]:
non_preq_issues = []
for i in issues:
    if not all_ranks[i]['preq']:
        #print(i)
        non_preq_issues.append(i)
len(non_preq_issues)

In [ ]:
# for now we'll only consider calls without issues

In [151]:
no_issues_ranks = [j for i, j in enumerate(all_ranks) if i not in issues]


In [48]:
len(all_ranks)

585

In [51]:
len(no_issues_ranks)

581

In [ ]:
no_issues_ranks[:2]

In [42]:
[i for i in range(0, len(no_issues_ranks)) if not 'docs_api' in no_issues_ranks[i].keys()]

[327, 393, 459, 460]

In [44]:
len(issues)

0

In [124]:
def compute_dcg_local(rank_results, i, verbose=False, api=False):
    scores = dict([(r['slug'], int(r['count'])) for r in rank_results[i]['docs']])
    is_preq = rank_results[i]['preq']
    occ = sum([c for (_,c) in scores.items()])
    if not api:
        y_pred = [c['slug'] for c in rank_results[i]['docs']]
    elif api:
        y_pred = rank_results[i]['docs_api']
    y_true = [c['slug'] for c in sort_ranks(rank_results[i]['docs'])]
    ndcg = discounted_cumulative_gain(scores, y_true, y_pred, 7)
    if verbose :
        print(json.dumps(scores, indent=2))
        print(json.dumps(y_pred, indent=2))
        print(json.dumps(y_true, indent=2))
        print(ndcg)
    return {'dcg': ndcg, 'occ': occ, 'preq': is_preq}

In [154]:
# retest all

j = 0
issues = []
#for i in range(0, len(all_ranks)):
#    if not control_call(i):
#        issues.append(i)
len(issues)
all_scores = pd.DataFrame([compute_dcg_local(no_issues_ranks, i, api=False) for i in range(len(no_issues_ranks))])
all_scores['weighted_dcg'] = all_scores['dcg'] * all_scores['occ'] 

preq_scores = all_scores[all_scores.preq]
other_scores = all_scores[~all_scores.preq]

print(f"nDCG all : {compute_weighted_dcg(all_scores)}")
print(f"nDCG preq : {compute_weighted_dcg(preq_scores)}")
print(f"nDCG other : {compute_weighted_dcg(other_scores)}")

nDCG all : 0.94
nDCG preq : 0.96
nDCG other : 0.88


In [142]:
# TODO : we need to ponderate DCGs on count (as DCG of a frequent query should be higher than other)

In [157]:
all_scores_api = pd.DataFrame([compute_dcg_local(no_issues_ranks, i, api=True) for i in range(len(no_issues_ranks))])
all_scores_api['weighted_dcg'] = all_scores_api['dcg'] * all_scores_api['occ'] 
all_scores["weighted_api"] = all_scores_api["weighted_dcg"]
all_scores["dcg_api"] = all_scores_api["dcg"]

In [173]:
all_scores[~all_scores.preq].sort_values(by=['occ', 'dcg'], ascending=False)[:50]

,dcg,occ,preq,weighted_dcg,weighted_api,dcg_api
559,0.999048,272,False,271.741011,271.741011,0.999048
504,0.990166,237,False,234.669372,228.603584,0.964572
194,0.994082,159,False,158.059096,158.059096,0.994082
360,0.962193,144,False,138.555847,138.555847,0.962193
229,0.992436,141,False,139.933418,139.933418,0.992436
454,0.999147,139,False,138.881447,138.881447,0.999147
467,0.997318,128,False,127.656675,124.428577,0.972098
50,0.812437,122,False,99.117338,99.117338,0.812437
212,0.986007,120,False,118.320898,118.320898,0.986007
81,0.998387,99,False,98.840354,98.840354,0.998387


In [169]:
all_scores[~all_scores.preq]['occ'].sum()

6261

In [170]:
all_scores[all_scores.preq]['occ'].sum()

22224

In [167]:
id = 269
no_issues_ranks[id]

{'total': 30,
 'queries': ['Rtt'],
 'preq': False,
 'docs': [{'slug': '/fiche-ministere-travail/risques-chimiques-reach#Pour-en-savoir-plus',
   'count': 1,
   'preq': 'semantic'},
  {'slug': '/fiche-service-public/reduction-du-temps-de-travail-rtt',
   'count': 29,
   'preq': 'both'},
  {'slug': '/fiche-service-public/representant-de-la-section-syndicale-rss',
   'count': 0,
   'preq': 'semantic'},
  {'slug': '/fiche-service-public/absence-dun-salarie-pour-cause-dintemperie-quelles-sont-les-regles',
   'count': 0,
   'preq': 'fulltext'},
  {'slug': '/fiche-service-public/un-salarie-en-formation-en-dehors-du-temps-de-travail-est-il-remunere',
   'count': 0,
   'preq': 'fulltext'},
  {'slug': 'https://www.telerc.travail.gouv.fr/RuptureConventionnellePortailPublic/jsp/site/Portal.jsp?page_id=20',
   'count': 0,
   'preq': 'semantic'},
  {'slug': '/fiche-service-public/greve-des-transports-quels-droits-pour-le-salarie',
   'count': 0,
   'preq': 'fulltext'}],
 'docs_api': ['/fiche-service

In [163]:
all_scores.loc[id]

dcg              0.93679
occ                   95
preq               False
weighted_dcg     88.9951
weighted_api      78.547
dcg_api         0.826811
Name: 78, dtype: object

In [165]:
compute_dcg_local(no_issues_ranks, id, verbose=True, api=True)

{
  "/fiche-ministere-travail/cse-attributions": 22,
  "/fiche-ministere-travail/cse-fonctionnement-et-moyens-dactions": 28,
  "/fiche-ministere-travail/cse-information-et-consultation": 13,
  "/fiche-ministere-travail/cse-attributions-du-cse-en-matiere-de-sante-de-securite-et-de-conditions-de-travail": 4,
  "/fiche-ministere-travail/cse-information-et-consultation#Quel-est-le-processus-de-consultation": 10,
  "/fiche-ministere-travail/cse-definition-cadre-et-calendrier-de-mise-en-place#Le-CSE-obligatoire-dans-quelles-entreprises": 10,
  "/fiche-ministere-travail/cse-commissions-sante-securite-et-conditions-de-travail-cssct": 8
}
[
  "/fiche-ministere-travail/cse-attributions",
  "/fiche-ministere-travail/cse-fonctionnement-et-moyens-dactions",
  "/fiche-ministere-travail/cse-information-et-consultation",
  "/fiche-ministere-travail/cse-information-et-consultation#Quel-est-le-processus-de-consultation",
  "/fiche-service-public/quels-sont-les-moyens-daction-du-comite-social-et-economiq

{'dcg': 0.8268108163164881, 'occ': 95, 'preq': False}

In [18]:

def compute_weighted_dcg(scores):
    return round(scores['weighted_dcg'].sum() / scores['occ'].sum(), 2)


In [61]:
preq_scores = all_scores[all_scores.preq]
other_scores = all_scores[~all_scores.preq]

print(f"nDCG all : {compute_weighted_dcg(all_scores)}")
print(f"nDCG preq : {compute_weighted_dcg(preq_scores)}")
print(f"nDCG other : {compute_weighted_dcg(other_scores)}")

nDCG all : 0.9
nDCG preq : 0.93
nDCG other : 0.79


In [ ]:
preq_scores = all_scores[all_scores.preq]
other_scores = all_scores[~all_scores.preq]

print(f"nDCG all : {compute_weighted_dcg(all_scores)}")
print(f"nDCG preq : {compute_weighted_dcg(preq_scores)}")
print(f"nDCG other : {compute_weighted_dcg(other_scores)}")

In [63]:
# index of ranks with pre-qualified
# index of ranks without

preq_indices = [i for i in range(0, len(no_issues_ranks)) if no_issues_ranks[i]['preq']]
other_indices = [i for i in range(0, len(no_issues_ranks)) if not no_issues_ranks[i]['preq']]

In [65]:
print(len(preq_indices))
print(len(other_indices))

265
316


In [ ]:
#preq_scores = [compute_dcg_local(no_issues_ranks, i) for i in preq_indices]
#other_scores = [compute_dcg_local(no_issues_ranks, i, api=False) for i in other_indices]

In [ ]:
#all_scores = [compute_metrics(no_issues_ranks, i, api=True) for i in range(len(no_issues_ranks))]

In [ ]:
# compute weighted average
# show : all / preq only / other / fulltex / sem

In [ ]:
# show : only fulltext / only sem

In [ ]:
#pd.DataFrame(all_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(all_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(preq_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(other_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(preq_scores + other_scores)['dcg'].describe()

In [28]:
run_preq = False
if run_preq:
    scores = preq_scores
    indices = preq_indices
else:
    scores = other_scores
    indices = other_indices

min_score = 0.98
min_occ = 5

to_check_indices = [(i, round(scores.iloc[i]['dcg'], 2), scores.iloc[i]['occ']) for i in range(0, len(scores)) if scores.iloc[i]['dcg'] < min_score and scores.iloc[i]['occ'] > min_occ]


In [31]:
# sanity check
test_i = to_check_indices[10][0]
pi = indices[test_i]
print(scores.iloc[test_i])
print(json.dumps(no_issues_ranks[pi], indent=2))

dcg             0.65413
occ                 122
preq              False
weighted_dcg    79.8039
Name: 50, dtype: object
{
  "total": 122,
  "queries": [
    "temps partiel",
    "TEMPS PARTIEL"
  ],
  "preq": false,
  "docs": [
    {
      "slug": "/fiche-ministere-travail/5-questions-reponses-sur-le-passage-a-temps-partiel#Question-1-Passage-a-temps-partiel-comment-obtenir-un-temps-partiel-nbsp",
      "count": 24,
      "preq": "both"
    },
    {
      "slug": "/fiche-service-public/temps-partiel-dans-le-secteur-prive",
      "count": 53,
      "preq": "fulltext"
    },
    {
      "slug": "/fiche-ministere-travail/5-questions-reponses-sur-le-passage-a-temps-partiel#Question-5-Passage-a-temps-partiel-quels-droits-pour-les-salaries-a-temps-nbsp",
      "count": 11,
      "preq": "both"
    },
    {
      "slug": "/fiche-service-public/temps-partiel-quelle-incidence-sur-les-droits-a-conges-payes-du-salarie",
      "count": 11,
      "preq": "fulltext"
    },
    {
      "slug": "/fich

In [27]:
def format_entry(i, queries, candidates_rank, score, count):
    lines = []
    lines.append(f"## ({i}) {list(queries)[0]}")
    lines.append(f'Score : `{score}`')
    lines.append(f'Count : `{count}`\n')
    lines.append('Variantes :')
    lines.extend(['- ' + q for q in queries])
    lines.append('')
    lines.append('|count|slug|algo|')
    lines.append('|--|--|--|')
    lines.extend([f"{ar['count']} | {ar['slug']} | {ar['preq']}" for ar in candidates_rank['docs']])
    lines.append('\n')
    #lines.extend([f"{ar}" for ar in candidates_rank['docs_api']])
    lines.append('\n\n')
    return lines



all_lines = []

# sort by count
to_check_indices.sort(key=lambda x: x[2], reverse=True)

for j in range(len(to_check_indices)) :
    (i, score, count) = to_check_indices[j]
    candidates_rank = no_issues_ranks[indices[i]]
    queries = candidates_rank['queries']
    
    all_lines.extend(format_entry(j, queries, candidates_rank, score, count))    

#print("\n".join(all_lines))
#print(all_lines)
#[print(f"{ar['count']} => {ar['slug']}") for ar in candidates_rank]

In [ ]:
with open("df-updates2.md", 'w+') as f :
    f.write("\n".join(all_lines))

In [ ]:
with open("search-issues-api2.md", 'w+') as f :
    f.write("\n".join(all_lines))

In [ ]:
all_ranks[51]

# Check best semantic entries

In [219]:
def sem_best(i):
    docs = no_issues_ranks[i]['docs']
    docs.sort(key=lambda x: x['count'], reverse=True)
    if (len(docs) > 1):
        return (docs[0]['preq'] == 'semantic') # or (docs[1]['preq'] == 'semantic')
    else :
        return False

sem_matches = [i for i in range(0, len(no_issues_ranks)) if sem_best(i)]

In [220]:
len(sem_matches)

45

In [248]:
[(no_issues_ranks[i]['queries'][0], no_issues_ranks[i]['docs'][0]['count'], call_api(no_issues_ranks[i]['queries'][0])['documents'][0]['_score']) for i in sem_matches]

[('prime macron', 11, 1.1565083),
 ('visite médicale', 8, 1.188532),
 ('invalidite', 2, 1.2326245),
 ('horaire variable', 5, 1.2501824),
 ('Penibilite', 2, 1.0760893),
 ('heures de délégation', 16, 1.2248789),
 ('heures de délégation', 29, 1.2248789),
 ('apprentissage', 20, 1.1658975),
 ('temps de repos légal', 3, 1.2196125),
 ('duree du travail', 21, 1.1322243),
 ('annualisation', 5, 1.2157235),
 ('13ème mois', 11, 1.1100206),
 ('licenciement motif personnel', 3, 1.5411197),
 ('discriminations', 25, 1.223724),
 ('grossesse', 5, 1.1548659),
 ('licenciement salarie protege', 3, 1.599219),
 ('refus formation', 2, 1.2625246),
 ('salaire apprenti', 9, 1.4086474),
 ('arrêt maladie et rupture conventionnelle', 3, 1.5381777),
 ('rupture conventionnelle et maladie', 8, 1.3388776),
 ('faute', 5, 1.0646229),
 ('vae', 3, 0),
 ('abandon de poste cdi', 18, 1.4859703),
 ('referent harcelement', 12, 1.183382),
 ('quels sont les motifs de licenciement', 3, 1.5063623),
 ('qui doit faire les entretiens 

In [231]:
no_issues_ranks[211]

{'total': 8,
 'queries': ['absence'],
 'preq': False,
 'docs': [{'slug': '/outils/preavis-demission',
   'count': 3,
   'preq': 'semantic'},
  {'slug': '/fiche-service-public/absence-dun-salarie-pour-cause-dintemperie-quelles-sont-les-regles',
   'count': 2,
   'preq': 'fulltext'},
  {'slug': '/fiche-service-public/labsence-du-salarie-est-elle-prise-en-compte-pour-le-calcul-de-ses-conges',
   'count': 2,
   'preq': 'fulltext'},
  {'slug': '/fiche-ministere-travail/les-conges-et-absences-pour-enfant-malade',
   'count': 1,
   'preq': 'semantic'},
  {'slug': '/fiche-ministere-travail/les-absences-liees-a-la-maladie-ou-a-laccident-non-professionnel',
   'count': 0,
   'preq': 'semantic'},
  {'slug': '/fiche-service-public/labsence-de-lemployeur-met-elle-fin-au-contrat-de-travail-du-salarie-a-domicile',
   'count': 0,
   'preq': 'fulltext'},
  {'slug': '/fiche-ministere-travail/les-absences-liees-a-la-maladie-ou-a-laccident-non-professionnel#Peut-il-y-avoir-licenciement-pour-maladie',
   '